# Lecture 1

In [0]:
import pandas as pd
import numpy as np
!pip install plotly==4.0.0
import plotly.express as px
import plotly.graph_objects as go


! if [ ! -e kirby127a_3_1_ax_283Labels_M2_corrected_stats.csv ] ; \
  then wget https://raw.githubusercontent.com/bcaffo/ds4bme_intro/master/data/kirby21.csv; \
fi
  
data = pd.read_csv("/content/kirby21.csv")

--2019-09-06 21:57:49--  https://raw.githubusercontent.com/bcaffo/ds4bme_intro/master/data/kirby21.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1611966 (1.5M) [text/plain]
Saving to: ‘kirby21.csv.3’

kirby21.csv.3       100%[===================>]   1.54M  --.-KB/s    in 0.05s   

2019-09-06 21:57:49 (29.5 MB/s) - ‘kirby21.csv.3’ saved [1611966/1611966]



## Question 1
Fit a model to the kirby 21 dataset into colab. Do a scatterplot of the left telencephalon by the right.

In [0]:
data1 = pd.read_csv("/content/kirby21.csv").drop(['Unnamed: 0', 'rawid', 'min', 'max', 'mean', 'std'], axis=1)
data2 = pd.DataFrame()

dataL = data1.loc[(data.roi == "Telencephalon_L")]
dataR = data1.loc[(data.roi == "Telencephalon_R")]

data2['VolL']=dataL["volume"].values
data2['VolR']=dataR["volume"].values

data2

fig = px.scatter(data2, x="VolR",y="VolL")
fig.show()

## Question 2
Create a vector of the left telencelphalon values and the right. Demean each. Fit a regression through the origin model with the left telecephalon as the outcome and the right as the predictor. Also do the same with the left as the predictor and the right as the outcome. Plot the fitted lines on your scatterplot. To obtain the regression slope, both code it yourself and use a regression function in python.

In [0]:
volL=dataL["volume"].values
volR=dataR["volume"].values

# demeaned vectors
volLDM=volL - volL.mean()
volRDM=volR - volR.mean()

# manually obtained regression model
def estimate_coeff(x, y):
  n = np.size(x)
  
  mean_x, mean_y = np.mean(x), np.mean(y)
  
  SS_xy = np.sum(y*x) - n*mean_y*mean_x
  SS_xx = np.sum(x*x) - n*mean_x*mean_x
  
  b_1 = SS_xy/SS_xx
  b_0 = mean_y - b_1*mean_x
  
  return(b_0, b_1)

mymodel = estimate_coeff(volL, volR)
mxmodel = estimate_coeff(volR, volL)

# using sklearn's linear regression model
from sklearn.linear_model import LinearRegression

x=volL.reshape((-1, 1))
y=volR

ymodel = LinearRegression().fit(x,y)
y_new = ymodel.predict(x)

x=volR.reshape((-1, 1))
y=volL

xmodel = LinearRegression().fit(x,y)
x_new = xmodel.predict(x)

# comparing manual and sklearn's models

print("Right Predictor model (int, slope)")
print("Manual Model")
print(mymodel)
print("sklearn Model")
print("(",ymodel.intercept_, ",", ymodel.coef_,")")

print("\n\nLeft Predictor model (int, slope)")
print("Manual Model")
print(mxmodel)
print("sklearn Model")
print("(",xmodel.intercept_, ",", xmodel.coef_,")")


fig = go.Figure()
fig.add_trace(go.Scatter(x=dataL["volume"].values, y=dataR["volume"].values,
                    mode='markers', name='Data'))
fig.add_trace(go.Scatter(x=dataL["volume"].values, y=y_new,
                    line=dict(dash='dot'),
                    name='Right Predictor'))
fig.add_trace(go.Scatter(x=x_new, y=dataR["volume"].values,
                    line=dict(dash='dot'),
                    name='Left Predictor'))
fig.update_layout(xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text="Left")),
                  yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text="Right")))

fig.show()

Right Predictor model (int, slope)
Manual Model
(-13875.944799423509, 1.0442415398322407)
sklearn Model
( -13875.944799423742 , [1.04424154] )


Left Predictor model (int, slope)
Manual Model
(17150.431045804056, 0.9500552111008129)
sklearn Model
( 17150.43104580429 , [0.95005521] )


## Question 3
Consider a regression through the origin model $Y=BX$. What would be the impact on the estimate of $B$ if we replaced $X$ by $cX$ where $c$ is a positive constant?

From definition, we have

$$ \hat\beta=\frac{\sum y_i x_i}{||x||} = \frac{\sum y_i x_i}{\sum x_i^2}$$

When $X$ is replaced by $cX$, we obtain
$$ \hat\beta_{new}=\frac{\sum y_i cx_i}{\sum (cx_i)^2}=\frac{c}{c^2}\frac{\sum y_i x_i}{\sum x_i^2}=\frac{1}{c}\hat\beta$$


Thus, we obtain
$$B_{new}=\frac{B}{c}$$


However, we also note that the model's prediction remains unchanged:
$$Y_{new}=B_{new}X_{new}=\frac{B}{c} cX=BX\equiv Y$$